In [12]:
import numpy as np
import os
import pandas as pd
import pickle
import string

from bisect import bisect_left, bisect_right
from collections import Counter, defaultdict
from datasets import Dataset, load_dataset, load_from_disk
from tqdm import tqdm

In [13]:
HUGGINGFACE_TOKEN = os.environ.get("HUGGINGFACE_TOKEN")

In [17]:
c4_small = load_dataset("datablations/small-c4-dedup", use_auth_token=HUGGINGFACE_TOKEN)["train"]

Using custom data configuration datablations--small-c4-dedup-2570b4ff8dac1dea


Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 419.68it/s]


Dataset parquet downloaded and prepared to /home/piktus_huggingface_co/.cache/huggingface/datasets/datablations___parquet/datablations--small-c4-dedup-2570b4ff8dac1dea/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 96.59it/s]


In [18]:
c4_small_df = pd.DataFrame(c4_small)

In [22]:
c4_small_df[c4_small_df["dup_ratio"] != 0.0]

,text,timestamp,url,meta,text_length,domain,perplexity,dup_ratio,pairs,repetitions,cluster,perplexity_score,included_in_dedup
19,Quartz is one of the most common and varied mi...,2019-04-19T01:04:23Z,None,{'perplexity_score': 345.8},7522,None,345.8,0.093326,"[[33651, 33779], [33989, 34109], [34769, 34872...",[b'm a deep sky blue color. Blue Quartz may al...,[19],345.8,True
54,Know Buckeye Trail Class of 0000 graduates tha...,2019-04-20T06:58:30Z,None,{'perplexity_score': 702.7},556,None,702.7,0.773381,"[[88777, 88914], [88924, 89067], [89080, 89230]]",[b'l Class of 0000 graduates that are NOT on t...,"[36112, 20290, 54, 98502]",702.7,True
86,"Pre-Owned, AWD Titanium 0dr Crossover, Ebony i...",2019-04-19T17:19:03Z,None,{'perplexity_score': 1213.5},5397,None,1213.5,0.047063,"[[175466, 175593], [178123, 178250]]","[b' interior, Gas, 00(city)/00(highwa\xc2\xady...",[86],1213.5,True
157,The White Russian is among the most popular dr...,2019-04-25T12:56:24Z,None,{'perplexity_score': 275.2},2241,None,275.2,0.433289,"[[316036, 316423], [316431, 316826], [317395, ...","[b""The White Russian is among the most popular...",[157],275.2,True
183,We focus on bringing the PANASONIC CGR-B/000A0...,2019-04-20T04:34:44Z,None,{'perplexity_score': 1555.8},2091,None,1555.8,0.267814,"[[367172, 367351], [367402, 367503], [367932, ...",[b'We focus on bringing the PANASONIC CGR-B/00...,[183],1555.8,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99933,You can have confidence in Asbestos Shingle Gu...,2019-04-19T04:24:10Z,None,{'perplexity_score': 455.9},2317,None,455.9,0.978420,"[[216156112, 216156217], [216156228, 216156735...",[b'\x01\x00You can have confidence in Asbestos...,"[99933, 56951]",455.9,True
99973,One claim constantly made about artistically i...,2019-04-22T18:07:34Z,None,{'perplexity_score': 259.1},13184,None,259.1,0.198726,"[[216217225, 216217329], [216219432, 216219536...",[b' The solution is increasing the demand whil...,[99973],259.1,True
99976,"As recently promised, I have finished a short ...",2019-04-22T04:20:27Z,None,{'perplexity_score': 362.1},388,None,362.1,0.314433,"[[216233617, 216233739]]",[b'. You can follow any responses to this entr...,"[99976, 6602, 4412]",362.1,True
99985,On this website we recommend many designs abao...,2019-04-18T10:19:12Z,None,{'perplexity_score': 780.4},743,None,780.4,0.547779,"[[216244436, 216244561], [216244600, 216244882]]",[b's that we have collected from various sites...,"[99985, 63410]",780.4,True


In [12]:
def find_whitespace(text):
    for i, c in enumerate(text):
        if c in string.whitespace:
            yield i

In [13]:
import csv
with open('/home/piktus_huggingface_co/lumi/oscar/queries.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    
    for idx, line in tqdm(enumerate(oscar_small)):
        text = line["text"]
        whitespace_idx = [-1] + list(find_whitespace(text))

        for i in whitespace_idx[::2]:
            if i + 101 < len(text):
                writer.writerow([text[i+1:i+101], idx])
    

    
"""
with open("/home/piktus_huggingface_co/lumi/oscar/queries.txt", "w") as queries:
    for idx, line in tqdm(enumerate(oscar_small)):
        text = line["text"]
        whitespace_idx = [-1] + list(find_whitespace(text))

        for i in whitespace_idx:
            if i + 101 < len(text):
                queries.write(text[i+1:i+101] + "\t" + str(idx) + "\n")
"""
# escape newlines

100000it [03:58, 419.39it/s]


'\nwith open("/home/piktus_huggingface_co/lumi/oscar/queries.txt", "w") as queries:\n    for idx, line in tqdm(enumerate(oscar_small)):\n        text = line["text"]\n        whitespace_idx = [-1] + list(find_whitespace(text))\n\n        for i in whitespace_idx:\n            if i + 101 < len(text):\n                queries.write(text[i+1:i+101] + "\t" + str(idx) + "\n")\n'

In [11]:
168694546 / 100000

1686.94546

In [3]:
file = np.fromfile("/home/piktus_huggingface_co/lumi/dedup/oscar_025/oscar.train", dtype=np.int32)

In [4]:
file_old = open("/home/piktus_huggingface_co/lumi/dedup/oscar_025/oscar.train", "rb").read()

In [17]:
file[100:120]

array([1886593133,  745370213, 1684955424, 1684368672, 1952539497,
       1936617321, 1752440876,  543519589,  543516788, 1702260589,
       1701344288, 1970565920, 1936025956, 1919907616,  543450475,
       1936682850, 1650532468,  544503151,  543516788, 1702129257],
      dtype=int32)

In [ ]:
file_byte = file.tobytes()

In [6]:
file_old[100:120]

b'e Pharmacy sildenafi'

In [8]:
len(file)

141238848310

In [9]:
len(file_old)

564955393242

In [5]:
c4_df = load_from_disk("/home/piktus_huggingface_co/lumi/preprocessed_data/c4-clusters/")

In [6]:
c4_df

Dataset({
    features: ['text', 'timestamp', 'url', 'perplexity_score', 'text_length', 'domain', 'dup_ratio', 'pairs', 'repetitions', 'cluster'],
    num_rows: 364868892
})

In [12]:
c4_df[11]

{'text': "Below you'll find some great videos that will encourage you, train you and build you up in hearing from GOD and being able to let HIM fulfill HIS plan in your life.\nSOMETHING NEW THAT WILL HELP YOU HEAR GODS VOICE!\nHow to Understand and Rightly Divide It & How It Applies to Life!\nIn this important teaching Terry reveals the clear distinction between the Spirit & Soul and how critical it is that we learn how to operate in the Spirit and not be deceived. It's a serious matter that must be reviewed no matter what your level of faith and maturity.\nMarriage & How it Applies to the Spirit & The Soul!\nIn this enlightening class Terry reveals a key insight that helps us to better discern the Spirit vs the Soul as it's reflected in the Marriage relationship from the intuitive nature, the emotional component of the woman and the power aspect of a male in submission to the LORD.\nIn this eye-opening teaching Terry shares how the Tabernacle as it represents the body soul and spirit 

In [15]:
for i in range(100):
    if len(c4_df[i]["pairs"]) > 0:
        print(i, c4_df[i])

4 {'text': 'The Denver Board of Education opened the 2017-18 school year with an update on projects that include new construction, upgrades, heat mitigation and quality learning environments.\nWe are excited that Denver students will be the beneficiaries of a four year, $572 million General Obligation Bond. Since the passage of the bond, our construction team has worked to schedule the projects over the four-year term of the bond.\nDenver voters on Tuesday approved bond and mill funding measures for students in Denver Public Schools, agreeing to invest $572 million in bond funding to build and improve schools and $56.6 million in operating dollars to support proven initiatives, such as early literacy.\nDenver voters say yes to bond and mill levy funding support for DPS students and schools. Click to learn more about the details of the voter-approved bond measure.\nDenver voters on Nov. 8 approved bond and mill funding measures for DPS students and schools. Learn more about what’s inclu

In [16]:
c4_df[4]

{'text': 'The Denver Board of Education opened the 2017-18 school year with an update on projects that include new construction, upgrades, heat mitigation and quality learning environments.\nWe are excited that Denver students will be the beneficiaries of a four year, $572 million General Obligation Bond. Since the passage of the bond, our construction team has worked to schedule the projects over the four-year term of the bond.\nDenver voters on Tuesday approved bond and mill funding measures for students in Denver Public Schools, agreeing to invest $572 million in bond funding to build and improve schools and $56.6 million in operating dollars to support proven initiatives, such as early literacy.\nDenver voters say yes to bond and mill levy funding support for DPS students and schools. Click to learn more about the details of the voter-approved bond measure.\nDenver voters on Nov. 8 approved bond and mill funding measures for DPS students and schools. Learn more about what’s include

In [18]:
for i in range(10000):
    if len(c4_df[i]["repetitions"]) > 0:
        for rep in c4_df[i]["repetitions"]:
            print(i, len(rep))

4 180
4 251
5 140
7 348
9 107
9 245
17 105
17 632
19 692
19 415
19 125
19 131
19 229
19 120
19 470
19 103
19 230
19 369
19 209
19 609
19 196
19 103
22 248
22 643
22 348
25 1938
26 227
26 108
26 127
26 123
26 191
26 164
28 147
28 283
28 100
28 483
28 128
30 325
30 588
30 218
30 296
30 172
35 100
35 271
35 235
36 135
41 112
41 875
41 129
42 297
44 185
46 152
46 130
46 229
46 202
46 253
54 294
54 156
55 328
56 234
57 197
57 187
57 214
57 145
58 433
62 102
62 197
63 379
63 724
63 336
63 552
63 368
63 154
63 153
63 146
63 948
63 210
63 216
63 399
63 308
63 112
63 361
63 790
63 637
63 135
63 688
63 295
63 359
63 141
63 113
63 476
63 214
63 111
63 243
63 173
63 153
63 153
63 179
63 235
63 149
63 379
63 132
63 104
63 874
63 222
63 101
63 127
63 294
63 285
63 112
63 181
63 235
63 1330
63 876
63 332
63 200
63 106
63 217
63 114
63 189
63 128
63 969
63 555
63 109
68 335
71 111
76 128
76 120
76 205
77 223
77 148
77 161
77 175
77 102
77 141
77 116
78 137
79 228
80 323
82 100
84 190
84 237
84 387
84 

1655 420
1655 497
1655 106
1656 122
1656 118
1657 104
1657 102
1657 153
1657 141
1657 424
1657 100
1657 114
1657 176
1659 471
1659 111
1661 118
1661 159
1662 280
1662 548
1662 562
1662 100
1662 110
1662 546
1662 251
1662 152
1662 336
1662 113
1662 131
1662 146
1662 721
1662 175
1663 229
1667 102
1667 205
1667 127
1668 141
1668 141
1673 111
1673 152
1674 204
1674 171
1674 120
1677 111
1677 126
1679 1383
1680 128
1683 128
1684 395
1684 534
1686 106
1687 653
1688 107
1689 167
1689 113
1689 444
1689 192
1689 796
1689 462
1690 341
1691 238
1691 109
1692 346
1692 114
1693 146
1693 428
1694 178
1694 835
1694 362
1698 136
1699 284
1699 2934
1702 129
1702 120
1703 113
1703 211
1706 102
1707 247
1707 252
1707 160
1707 173
1707 113
1708 227
1708 234
1708 156
1711 453
1711 110
1711 199
1712 868
1714 183
1715 268
1718 106
1718 272
1718 118
1718 940
1718 979
1718 1845
1718 164
1718 2128
1718 2133
1718 2118
1718 422
1718 737
1718 368
1718 267
1718 130
1718 745
1718 997
1718 1442
1718 141
1718 370
171

2476 2623
2478 199
2478 156
2478 116
2478 284
2478 267
2478 219
2480 238
2480 1858
2485 189
2485 192
2485 184
2485 275
2486 143
2486 148
2488 108
2489 306
2489 314
2491 123
2498 103
2503 124
2503 124
2506 576
2507 257
2507 195
2507 230
2507 336
2508 1534
2511 288
2513 208
2513 129
2513 414
2513 236
2513 137
2513 157
2514 322
2514 166
2514 264
2514 284
2514 134
2514 244
2514 139
2514 182
2515 278
2516 289
2516 209
2516 201
2520 383
2521 147
2523 171
2523 100
2523 329
2524 539
2525 177
2525 883
2525 196
2528 224
2531 170
2531 406
2533 528
2534 333
2534 522
2535 110
2535 110
2535 303
2541 162
2541 123
2541 123
2541 106
2541 135
2543 356
2545 233
2545 164
2545 100
2547 247
2547 188
2548 112
2550 367
2550 509
2553 144
2553 149
2553 177
2553 145
2556 467
2558 266
2566 169
2568 983
2570 824
2571 812
2571 156
2571 296
2572 104
2572 176
2572 123
2572 430
2572 164
2572 104
2572 263
2575 115
2576 133
2576 133
2578 140
2579 668
2579 178
2579 908
2579 928
2579 238
2579 125
2581 276
2581 488
2582 29

3302 160
3302 120
3303 101
3304 159
3304 134
3304 139
3304 124
3304 152
3304 978
3304 125
3304 181
3304 247
3304 200
3304 281
3304 695
3304 162
3304 193
3304 192
3304 528
3304 175
3304 120
3304 182
3304 140
3304 146
3304 834
3304 102
3304 479
3304 382
3304 491
3304 1120
3309 238
3309 111
3311 171
3311 246
3311 267
3311 170
3311 299
3311 185
3312 208
3312 219
3312 138
3312 693
3312 188
3312 165
3315 190
3316 136
3316 110
3316 258
3318 152
3319 741
3322 284
3323 2157
3325 102
3325 139
3326 440
3327 371
3330 252
3330 159
3331 170
3332 100
3332 115
3332 185
3332 102
3332 286
3332 167
3332 217
3332 142
3332 228
3332 299
3332 110
3332 186
3332 365
3332 308
3332 306
3332 468
3332 442
3332 150
3332 124
3332 552
3332 183
3332 216
3332 124
3332 590
3332 249
3332 472
3332 182
3332 116
3332 324
3332 459
3332 233
3332 215
3332 297
3332 188
3332 196
3332 120
3332 107
3332 538
3332 185
3332 144
3332 175
3332 141
3334 137
3334 158
3334 228
3335 113
3335 104
3343 826
3346 517
3346 201
3346 1508
3351 55

4144 431
4147 127
4147 131
4148 134
4148 151
4148 158
4152 124
4153 668
4158 493
4163 257
4163 261
4165 283
4165 262
4168 559
4168 117
4168 100
4174 391
4174 102
4178 161
4178 455
4179 195
4180 144
4180 424
4180 215
4186 170
4186 106
4187 304
4188 123
4192 132
4192 146
4194 245
4194 245
4201 118
4201 131
4201 206
4201 253
4201 119
4201 199
4201 102
4201 188
4204 112
4204 220
4209 190
4210 149
4210 159
4210 125
4210 123
4210 117
4212 134
4212 402
4212 367
4214 177
4214 103
4215 100
4216 121
4216 105
4216 110
4216 278
4216 102
4216 394
4216 559
4216 657
4218 101
4219 420
4219 145
4221 676
4221 110
4221 135
4221 118
4221 187
4223 197
4223 123
4223 182
4226 167
4226 119
4226 397
4226 193
4227 149
4228 244
4228 314
4232 508
4232 202
4233 142
4235 3058
4236 993
4236 198
4236 151
4236 105
4237 201
4237 379
4238 107
4238 100
4239 156
4239 165
4240 288
4242 126
4242 185
4249 229
4252 139
4252 459
4255 182
4256 100
4257 142
4257 384
4257 142
4257 148
4257 160
4257 1198
4257 270
4257 224
4257 111

...

5817 645
5819 106
5820 274
5820 173
5820 449
5820 358
5821 109
5824 116
5825 351
5825 162
5826 163
5826 148
5831 154
5831 872
5832 200
5832 212
5834 149
5835 304
5836 168
5836 123
5837 146
5837 139
5837 238
5837 435
5837 232
5837 120
5837 399
5837 620
5840 118
5842 404
5842 140
5844 140
5845 108
5846 153
5848 608
5849 111
5849 239
5849 184
5853 2699
5853 1142
5854 223
5854 113
5855 104
5855 308
5855 764
5857 104
5862 117
5865 122
5865 155
5865 148
5865 128
5867 398
5867 434
5867 453
5867 438
5867 765
5867 816
5867 282
5867 291
5867 682
5867 844
5867 456
5867 728
5867 252
5867 674
5867 478
5867 497
5867 801
5867 573
5867 158
5869 153
5870 117
5870 631
5870 128
5870 204
5873 303
5873 179
5873 133
5873 114
5873 214
5873 502
5876 706
5876 123
5879 192
5879 243
5879 116
5879 211
5879 203
5879 116
5879 101
5879 328
5879 101
5879 277
5879 148
5879 166
5879 211
5879 143
5881 149
5881 113
5881 114
5881 139
5881 106
5883 193
5884 142
5884 280
5884 1673
5885 244
5885 172
5885 385
5885 128
5888 18

6654 149
6657 125
6657 252
6657 134
6658 143
6658 298
6658 127
6662 145
6662 151
6662 136
6662 216
6664 356
6665 182
6665 112
6665 150
6665 200
6665 238
6667 119
6667 111
6667 108
6667 228
6667 284
6668 132
6669 254
6675 248
6676 104
6676 566
6676 314
6679 626
6679 530
6679 624
6679 307
6686 474
6686 196
6687 107
6687 181
6687 118
6688 100
6688 100
6688 118
6688 155
6690 101
6690 523
6692 186
6692 171
6692 206
6693 266
6696 240
6700 216
6702 318
6702 169
6704 348
6704 246
6708 139
6708 215
6708 258
6708 577
6709 349
6709 101
6709 178
6709 145
6709 182
6709 217
6709 117
6715 236
6715 101
6715 326
6715 283
6715 357
6715 301
6715 135
6715 348
6715 400
6717 129
6717 149
6719 152
6719 158
6719 120
6719 143
6721 160
6721 161
6722 1620
6726 248
6730 7507
6736 104
6738 301
6739 101
6739 402
6739 231
6740 353
6740 899
6740 859
6742 100
6745 114
6750 308
6752 288
6752 391
6756 157
6756 188
6756 170
6756 180
6756 144
6756 140
6757 303
6757 162
6760 296
6761 112
6767 181
6767 281
6767 691
6767 102

7558 270
7559 380
7559 218
7561 123
7561 150
7563 120
7563 121
7569 107
7569 109
7572 346
7572 287
7572 666
7572 337
7572 157
7573 142
7573 219
7573 108
7574 359
7581 141
7584 168
7584 428
7584 2768
7584 1672
7584 3634
7584 166
7587 267
7589 101
7589 199
7589 158
7589 239
7589 122
7591 934
7592 116
7592 648
7592 179
7594 150
7594 297
7594 312
7594 227
7594 115
7594 138
7594 393
7594 105
7594 403
7594 167
7594 182
7594 104
7594 159
7594 102
7594 516
7594 199
7594 308
7594 417
7594 148
7594 197
7594 139
7596 108
7598 205
7599 117
7599 195
7602 473
7602 368
7602 114
7602 408
7602 116
7609 101
7611 201
7612 201
7613 139
7613 187
7616 630
7621 122
7621 142
7621 111
7621 141
7621 130
7621 135
7621 107
7621 176
7621 120
7621 150
7621 191
7621 273
7621 144
7621 121
7621 103
7622 138
7629 124
7629 148
7629 218
7629 107
7629 242
7633 487
7633 865
7635 115
7637 279
7637 104
7638 851
7642 854
7642 1200
7642 334
7642 234
7642 188
7642 188
7645 550
7645 354
7645 255
7648 641
7651 474
7651 241
7651 1

8409 408
8409 175
8409 102
8409 100
8409 236
8409 172
8410 130
8412 329
8414 106
8414 106
8415 635
8415 142
8415 247
8416 1372
8416 152
8416 462
8416 112
8416 109
8416 126
8416 716
8417 929
8422 379
8423 304
8423 112
8424 133
8427 113
8428 310
8428 227
8428 310
8428 227
8428 137
8431 261
8431 510
8435 315
8437 211
8437 120
8442 211
8442 197
8442 420
8444 687
8444 849
8444 131
8447 195
8447 561
8447 122
8448 293
8449 125
8449 171
8449 136
8449 125
8449 437
8449 148
8449 630
8449 454
8449 117
8449 122
8449 236
8449 254
8449 129
8449 109
8450 160
8452 221
8452 104
8452 150
8453 110
8456 176
8456 129
8459 124
8461 2476
8462 120
8462 135
8462 120
8462 135
8462 121
8462 130
8462 101
8463 106
8463 142
8465 201
8471 504
8475 105
8475 108
8476 224
8477 269
8478 232
8478 411
8478 291
8478 149
8478 119
8478 128
8478 109
8478 170
8478 176
8478 298
8478 143
8478 116
8478 131
8478 135
8478 106
8478 379
8478 301
8478 349
8478 110
8478 190
8478 116
8478 164
8478 270
8478 222
8478 133
8478 135
8478 282

9243 656
9243 441
9243 257
9247 246
9249 101
9251 702
9252 105
9252 110
9253 117
9253 117
9256 727
9256 144
9256 428
9256 1020
9256 202
9256 103
9256 162
9256 455
9256 130
9257 707
9259 281
9259 289
9260 351
9262 112
9263 274
9263 258
9263 338
9263 123
9263 195
9268 159
9268 146
9268 327
9268 377
9268 344
9268 154
9268 302
9268 186
9268 538
9268 251
9268 204
9274 521
9275 215
9275 127
9275 102
9275 125
9281 101
9281 181
9281 101
9282 271
9283 1463
9287 204
9287 215
9287 212
9287 291
9288 110
9289 224
9291 231
9291 225
9292 118
9292 112
9292 121
9292 160
9293 107
9293 105
9293 106
9300 109
9302 257
9308 967
9308 1888
9308 798
9308 1239
9308 332
9308 158
9308 319
9310 459
9314 1276
9318 189
9318 239
9318 1649
9318 169
9319 210
9320 154
9320 129
9320 112
9320 156
9320 101
9320 111
9322 262
9324 138
9325 390
9328 390
9329 399
9331 138
9331 134
9331 237
9331 138
9333 379
9336 155
9336 155
9337 125
9337 201
9337 229
9337 1505
9345 119
9345 137
9348 308
9348 149
9351 138
9353 124
9354 130
935